### Notebook to clean up ethograms to be added to ground truth

In [1]:
import pandas as pd
from ipywidgets import HBox, VBox, Select, Button, Layout, RadioButtons
from fastplotlib import ImageWidget, Plot
from mesmerize_core.arrays import LazyVideo
from pathlib import Path
import numpy as np
from fastplotlib.graphics.selectors import LinearRegionSelector, LinearSelector

2023-06-14 12:12:35.167940: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 12:12:35.614356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ETHOGRAM_COLORS = {
    "lift": "b",
    "handopen": "green",
    "grab": "r",
    "sup": "cyan",
    "atmouth": "magenta",
    "chew": "yellow"
}

### dataframe to store cleaned ethograms

In [7]:
# clean_df = pd.DataFrame(columns=["trial", "cleaned_ethogram"])
# clean_df.to_hdf('/data/caitlin/cleaned_ethograms.hdf', key='df')

clean_df = pd.read_hdf('/data/caitlin/cleaned_ethograms.hdf')
clean_df

,trial,cleaned_ethogram


### load dataframe that marks which trials to be cleaned

In [8]:
mark_trials_df = pd.read_hdf('/data/caitlin/mark_trials.hdf')
mark_trials_df = mark_trials_df[mark_trials_df['mark'] == "clean"]
mark_trials_df

,trial,mark
0,M232_20170306_v024,clean
1,M232_20170306_v025,clean
3,M232_20170306_v074,clean
4,M232_20170306_v075,clean
5,M232_20170306_v076,clean
7,M232_20170308_v028,clean
9,M232_20170308_v034,clean


In [9]:
options = [trial for trial in mark_trials_df["trial"]]

### get original merged ethograms that need to be cleaned

In [10]:
ethograms_df = pd.read_hdf("/data/caitlin/exactly1_hand_labels.hdf")

In [11]:
ethograms = dict()

In [12]:
for row in ethograms_df.iterrows():
    keys = row[1]['merged_ethogram'].keys()
    for k in keys:
        ethograms[k] = row[1]['merged_ethogram'][k]

In [13]:
data_path = Path('/data/caitlin/potential_ground_truth/')

In [14]:
def trial_change(obj):
    selected_vid = data_path.joinpath(trial_selector.value).with_suffix('.avi')
    iw.set_data([LazyVideo(selected_vid)], reset_vmin_vmax=True)
    plot.clear()
    make_plot(plot)
    plot.auto_scale()

In [15]:
def clean_ethogram(obj):
    # get indices of selected region
    selected_ixs = plot.selectors[1].get_selected_indices(plot.graphics[0])
    # map behavior button value to index
    behavior_ix = behavior_buttons.options.index(behavior_buttons.value)
    # set indices of selected region to 
    if fill_values.value == "0":
        plot.graphics[behavior_ix].colors[selected_ixs[0]:selected_ixs[-1]] = "black"
    else:
        plot.graphics[behavior_ix].colors[selected_ixs[0]:selected_ixs[-1]] = ETHOGRAM_COLORS[behavior_buttons.value]

In [17]:
def save_new_ethogram(obj):
    # create new ethogram based off of indices that are not black
    trial_length = ethograms[trial_selector.value][0].shape[0]
    new_ethogram = np.zeros(shape=(6, trial_length))
    for i, graphic in enumerate(plot.graphics):
        non_zero_ixs = np.where(plot.graphics[i].colors[:] != np.array([0, 0, 0, 1]))[0]
        new_ethogram[i][non_zero_ixs] = 1
    # check if key in clean_df
    if trial_selector.value in trial_selector.value in list(clean_df["trial"]):
        ix = clean_df.loc[clean_df["trial"] == trial_selector.value].index[0]
        clean_df.loc[ix, "cleaned_ethogram"] = new_ethogram
    # else add to end of df
    else:
         clean_df.loc[len(clean_df.index)] = [trial_selector.value, new_ethogram]
    # set df_marked[trial_selector.value] = "good"
    ix = mark_trials_df.loc[mark_trials_df["trial"]==trial_selector.value].index[0]
    mark_trials_df.loc[ix, "mark"] = "good"
    mark_trials_df.to_hdf('/data/caitlin/mark_trials.hdf', key='df')
    # save clean_df to disk
    clean_df.to_hdf('/data/caitlin/cleaned_ethograms.hdf', key='df')

In [18]:
def reset_ethogram(obj):
    old_ethogram = ethograms[trial_selector.value]
    for i, graphic in enumerate(plot.graphics):
        graphic.colors[old_ethogram[i] == 1] = list(ETHOGRAM_COLORS.values())[i]
    ix = mark_trials_df.loc[mark_trials_df["trial"]==trial_selector.value].index[0]
    mark_trials_df.loc[ix, "mark"] = "clean"
    #[mark_trials_df["trial"] == trial_selector.value]["mark"] = "clean"
    mark_trials_df.to_hdf('/data/caitlin/mark_trials.hdf', key='df')

In [19]:
# radio buttons to click which behavior needs to be changed 
behavior_buttons = RadioButtons(options=["lift", "handopen", "grab", "sup", "atmouth", "chew"], layout=Layout(width='auto'))
# radio button to check whether the value should be changed to zero or 1
fill_values = RadioButtons(options=["0", "1"], layout=Layout(width='auto'))
# button to clean the ethogram based on the values of the radio buttons
clean_button = Button(value=False, disabled=False, icon='broom',
                                       layout=Layout(width='auto'), tooltip='clean ethogram')
clean_button.on_click(clean_ethogram)
# save button to save new ethogram to cleaned_df and save df to disk also change the trial in marked_df to "good"
save_button = Button(value=False, disabled=False, icon='save',
                                       layout=Layout(width='auto'), tooltip='save clean ethogram')
save_button.on_click(save_new_ethogram)
    # check if key exists, and overwrite, otherwise add
# reset button to reset the ethogram to what is stored in marked_df and set key of marked_df to "clean"
reset_button = Button(value=False, disabled=False, icon='history',
                                       layout=Layout(width='auto'), tooltip='reset ethogram')
reset_button.on_click(reset_ethogram)

In [20]:
radio_box = HBox([behavior_buttons, fill_values])
clean_options = HBox([reset_button, clean_button, save_button])

In [21]:
trial_selector = Select(options=options)
trial_selector.observe(trial_change, "value")

In [22]:
iw = ImageWidget(data=[LazyVideo(data_path.joinpath(trial_selector.value).with_suffix('.avi'))])

RFBOutputContext()

/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:87: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:92: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


In [23]:
def ethogram_event_handler(ev):
    ix = ev.pick_info["selected_index"]
    iw.sliders["t"].value = ix

In [24]:
plot = Plot(size=(500, 100))
def make_plot(plot):
        
    ethogram_array = ethograms[trial_selector.value]
    y_bottom = 0
    for i, b in enumerate(ETHOGRAM_COLORS.keys()):
        xs = np.arange(ethogram_array.shape[1], dtype=np.float32)
        ys = np.zeros(xs.size, dtype=np.float32)

        lg = plot.add_line(
            data=np.column_stack([xs, ys]),
            thickness=10,
            name=b
        )

        lg.colors = 0
        lg.colors[ethogram_array[i] == 1] = ETHOGRAM_COLORS[b]

        y_pos = (i * -10) - 1
        lg.position_y = y_pos

    ethogram_region_selector = LinearRegionSelector(
                bounds=(0, 50),
                limits=(0, ethogram_array.shape[1]),
                axis="x",
                origin=(0,-25),
                fill_color=(0, 0, 0, 0),
                parent=lg,
                size=(55),
            )
    
    ethogram_selector = LinearSelector(
                selection=0,
                limits=(0, ethogram_array.shape[1]),
                axis="x",
                parent=lg,
                end_points=(y_bottom, y_pos),
            )

    plot.add_graphic(ethogram_selector)
    plot.add_graphic(ethogram_region_selector)
    ethogram_selector.selection.add_event_handler(ethogram_event_handler)

RFBOutputContext()

In [25]:
make_plot(plot)
VBox([
    HBox([iw.show(), 
          VBox([
              trial_selector
              ])
            ]),
    HBox([plot.show(),
         VBox([radio_box, clean_options])
         ])
])

In [28]:
mark_trials_df

,trial,mark
0,M232_20170306_v024,clean
1,M232_20170306_v025,clean
3,M232_20170306_v074,good
4,M232_20170306_v075,clean
5,M232_20170306_v076,clean
7,M232_20170308_v028,clean
9,M232_20170308_v034,clean


In [29]:
clean_df

,trial,cleaned_ethogram
0,M232_20170306_v074,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
